# "Just in time GUIs" for interactive exploration

## Bridging JavaScript and the IPython kernel in the browser

In this exercise we illustrate IPython's interactive capabilities by building an image comparison tool, inspired by [Github's image diff](https://github.com/blog/817-behold-image-view-modes).

We will see how we can build, with a very limited amount of code, a small tool to assist us in comparing two related images visually, a task often needed in data analysis.  While working interactively with code, most of us probably simply will plot both next to each other and look at them side by side...

Since this example was written as an illustration for a talk at [Neil Lawrence's Data Science and Machine Learning initiative at Sheffield](http://staffwww.dcs.shef.ac.uk/people/N.Lawrence/), we'll use for our illustration the old and new version of the coat of arms for the University of Sheffield:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

a = plt.imread("University_of_Sheffield_coat_of_arms_old.png")
b = plt.imread("University_of_Sheffield_coat_of_arms_new.png")
plt.rcParams['figure.figsize'] = (8,6)
fig, (ax_a, ax_b) = plt.subplots(1, 2)
ax_a.imshow(a)
ax_b.imshow(b);

It would be great to be able to visualize the changes in various forms: as an image difference, blending the two, superimposing them... 

Let's build a set of comparison functions that will return a new image $c = f(a, b)$, so we can display the result. More generically, how we produce the comparison between $a$ and $b$ can depend on a parameter, say $\alpha$, so our comparison functions in general will be of the form $c = f(a, b, \alpha)$.

This is precisely the functionality offered by [Github's image diff tools](https://github.com/blog/817-behold-image-view-modes):

In [ ]:
def c_blend(a, b, alpha):
    """Blend images a and b.
    """
    return (1.0-alpha)*a + alpha*b


def c_diff(a, b, alpha):
    """Return the difference between a and b.
    """
    # Note: we still leave alpha as a parameter for reasons
    # so we can call this function with the same interface as the others
    return a-b


def c_horizontal_shade(a, b, alpha):
    """Form c from top rows of a and bottom rows of b.
    """
    nrows = a.shape[0]
    r_index = int(alpha*nrows)
    c = np.empty_like(a)
    c[:r_index] = a[:r_index]
    c[r_index:] = b[r_index:]
    return c


def c_vertical_shade(a, b, alpha):
    """Form c from left columns of a and right columns of b.
    """
    ncols = a.shape[1]
    c_index = int(alpha*ncols)
    c = np.empty_like(a)   
    c[:, :c_index] = a[:, :c_index]
    c[:, c_index:] = b[:, c_index:]
    return c

Now that we have defined all our specific comparison functions, we can call any of them via a wrapper utility function that selects which one to uses, and ensures that the color limits are always the actual limiting values of the orginal data. Let's define this utility:

In [ ]:
def compare(a, b, method=c_vertical_shade, alpha=0.5):
    """Show the comparison between images a and b using the 
    selected comparison method.
    """
    c = method(a, b, alpha)
    cmin = min(a.min(), b.min())
    cmax = max(a.max(), b.max())
    plt.imshow(c, clim=(cmin, cmax))

And using it, we can now call it as follows:

In [ ]:
compare(a, b, alpha=0.250)

Or for example:

In [ ]:
compare(a, b, c_horizontal_shade, 0.4)

# Exercise

Now, using the IPython `interact` facility, you should create a graphical interface to call `compare` with a slider for `alpha` between 0 and 1, and a selector for all the possible comparison methods.

In [ ]:
# your code here